In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag


In [2]:
# I have exported the relevant data into a new excel -  export3.xlsx, convert excel to pandas

df = pd.read_excel('data/export3.xlsx')

''' 
#Reading from excel files with multiple tabs.
xlsx = pd.ExcelFile('inc1.xlsx')

data_sheets = []
for sheet in xlsx.sheet_names:
    data_sheets.append(pd.read_excel(xlsx,sheet, usecols=[1,3,4,5,6,7,11,12,13,14,15,16,19,27]))
df = pd.concat(data_sheets)
df=df.reset_index(drop=True)
df.columns = [c.replace(' ', '_') for c in df.columns]
'''
print("Input Dataframe Shape(rows,cols):",df.shape)


Input Dataframe Shape(rows,cols): (13515, 14)


In [105]:
df1.Location.value_counts()

Corporate Office - Phoenix 1                                   28199
St Joseph's Hospital and Medical Center (SJHMC)                 6009
St Joseph's Medical Center Stockton (SJMC)                      2601
Chandler Regional Medical Center (CRMC)                         2455
Mercy San Juan Hospital (MSJMC)                                 2228
Bakersfield Memorial Hospital (BMH)                             2037
Northridge Hospital and Medical Center Roscoe (NHMC)            2025
St Rose Dominican Hospital - Siena (SRDH-S)                     1939
California Hospital Medical Center (CHMC)                       1895
Marian Regional Medical Center (MRMC)                           1865
St Mary Medical Center Long Beach (SMMC-LB)                     1855
St Johns Regional Medical Center (SJRMC)                        1792
St Bernardine Medical Center (SBMC)                             1673
Mercy General Hospital (MGH)                                    1670
Dominican Hospital (DSC)          

In [3]:
df.head()

,Customer,Priority,State,Short description,Assignment group,Category,Child Incidents,Close notes,Close code,Closure CI,Configuration item,Contact type,Created,Location
0,Rebecca Borghi,4 - Low,New,Access DH: Dominican facility not available as...,DH-GSSAPACSAdmin,Application,0.0,NaN,NaN,NaN,Access Dignity Health,Phone,2018-10-25 15:22:02,Dominican Hospital (DSC)
1,Logan McDaniel,4 - Low,New,"ECF update for Location Change for Burns, Josh...",DHE-HR Technology,Application,0.0,NaN,NaN,NaN,Lawson Employee and Manager Self Service 10.0.7,Phone,2018-10-31 00:36:25,St Rose Dominican Hospital - Siena (SRDH-S)
2,OMI Integration Service,2 - High,New,'mmc-vint-017.chw.edu': Ping packet good perce...,DHE-Server Build Engineering,Server,0.0,NaN,NaN,NaN,mmc-vint-017,Event,2018-10-27 15:28:02,Corporate Office - Phoenix 1
3,Madeline Richter,3 - Moderate,New,Allscripts || Unable to see any patients chart...,DH-BNI Informatics,Security,0.0,NaN,NaN,NaN,Application ID Issue,Phone,2018-10-29 08:20:20,St Joseph's Hospital and Medical Center (SJHMC)
4,OMI Integration Service,4 - Low,New,'bni-aio-001.chw.edu': Utilization is '85.17'....,DH-BNI Informatics,Server,0.0,NaN,NaN,NaN,bni-aio-001,Event,2018-11-05 15:00:47,Corporate Office - Phoenix 1


In [42]:
#Filter out some categories with instances <1000 for now , remove this filter in future
df1 = df.groupby("Category").filter(lambda x: len(x) >1000)
df1 = df1.reset_index(drop =True)
df1.Category.value_counts()

End User Services    22410
Application          20651
Security             17171
Server               12687
Network               7831
HelpDesk              5169
EAI Interface         3028
Telecom               2832
Human Resources       1224
Name: Category, dtype: int64

In [53]:

df1['newtext'] = df1.Short_description.map(str) +' '+ df1.Location.map(str)+' '+ df1.Customer.map(str)
#+' '+ df1.Contact_type.map(str)
# Can join other features since we need to just do clustering for now

In [63]:
#NLTK text preprocessor ---
#todo :try Spacy

stop = stopwords.words('english')
lemmer=WordNetLemmatizer()

#extend stop words
newStopWords = ['chw','edu','phx','vapp']
stop.extend(newStopWords)

#POS tagging(not used)
def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

def pre_process(X):
    pat = r'[^A-Za-z ]+'
    X = X.str.replace(pat, ' ',regex=True)
    X = X.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))   #stop
    X = X.apply(lambda x: ' '.join([lemmer.lemmatize(word) for word in x.split()]))       #lemma
    X = X.apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1 ]))  #length >1
   #X = X.apply(lambda x: ' '.join([word for word,pos in pos_tag(x.split()) if pos != 'NNP' and pos != 'NNPS']))

    return X


In [64]:
X2 = pre_process(df1['newtext'])

In [46]:
# define a tokenizer which returns the set of tokens in the text that it is passed


def tokenize_only(text):
    # tokenize by  word 
    tokens = [word.lower() for word in nltk.word_tokenize(text) ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

#  tf-idf

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer( max_features=10000,max_df=0.9,min_df=0.0001,                                  
                                  use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(X2.values.astype('U'))

print(tfidf_matrix.shape)

Wall time: 32.1 s
(95373, 10000)


In [101]:
#Get the ngram features before doing k-means
#filter out unnecessary terms and add to stop words
terms = tfidf_vectorizer.get_feature_names()
print(terms[150:350])

['access transaction', 'access transaction failed', 'access user', 'access user id', 'access user name', 'access username', 'access website', 'access without', 'access without issue', 'account able', 'account able access', 'account able login', 'account account', 'account account lock', 'account account locked', 'account activated', 'account activated issue', 'account activation', 'account active', 'account created', 'account creation', 'account customer', 'account customer able', 'account customer unable', 'account disabled', 'account disabled user', 'account disablement', 'account enable', 'account enable ms', 'account enabled', 'account enabled able', 'account expired', 'account first', 'account first time', 'account getting', 'account hduv', 'account hduv done', 'account issue', 'account issue account', 'account issue fixed', 'account issue need', 'account issue resolved', 'account issue unable', 'account issue user', 'account lock', 'account lock job', 'account lock network', 'acc

In [ ]:
df1.shape

# kmeans

In [102]:
#Mini Batch K means
from sklearn.cluster import MiniBatchKMeans
num_clusters = 50
km = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=3,init_size=3000, batch_size=1000)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

Wall time: 1.39 s


In [118]:
#Save cluster in pickle file for future use
import pickle

pickle.dump(km, open('pkl/km_1.pkl', 'wb'))

In [103]:
##Create mapping of ngram features into clusters, 
#save into terms map object with key as cluster name (first feature in each cluster)
#terms = top 20 ngram features

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
terms_map = pd.DataFrame(index=np.arange(0, num_clusters),columns=['terms','key'])    
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    #terms_map['key'] = i
    list = [] 
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind], end=',')
        list.append(terms[ind])
    for ind2 in order_centroids[i, :1]:
        terms_map.key.loc[i] = terms[ind2]

    str = ', '.join(list) 
    terms_map.terms.loc[i] = str
    print()
print(terms_map.shape)

Cluster 0: soh edu, sonifi health, connect socket, unable connect socket, health tier, application unable connect, point application unable, socket timeout, application unable, end point application, point application, thrown end point, end point, error thrown end, thrown end, socket timeout keep, alive error thrown, realtime socket, realtime socket timeout, timeout keep alive,
Cluster 1: hduv passed user, passed user able, passed user, login hduv passed, hduv passed, unable login hduv, login hduv, reset network unable, password reset network, network unable login, network password reset, network unable, reset network, network password, user able login, user able, enup active, user logged successfully, logged successfully, user logged,
Cluster 2: close yes inventory, yes inventory updated, yes inventory, close yes, customer approval, approval close yes, customer approval close, approval close, affected user name, affected user, user name, inventory updated, device name, inventory updat

In [104]:
#Save Terms Map  to csv , terms = Key =first instance of 
terms_map.to_csv('terms_map01.csv')
#print(terms_map)
#MANUAL STEP
#Update CSV : meaningful Cluster name for column 'key'. Index has cluster number
#0 -> login 


#Terms Map CSV Manual Work
#Key is the first instance of each term list, Manually update to make it meaningful

             terms	                                         key             (new) key
0	login user, login user id, login, unable login...	login user      ---> login pwd 
1	cerner, issue, unable, ms, id, login, user, ac...	cerner          ---> cerner
2	dragon, dragon medical, dragon application iss...	dragon          ---> application issue
3	physician issue, cerner physician issue, cerne...	physician issue ---> cerner 
4	job completed, active job, completed exit, com...	job completed   ---> job status

In [86]:
#Read CSV again
terms2=pd.read_csv('terms_map01.csv')
#Convert Keys to Dict, with index mapped into cluster name
dict2 = terms2.key.to_dict()


In [87]:
#dataframe add new column 'cluster' for the cluster numbers (int)
df1['cluster'] = km.labels_
#dataframe map integer 'cluster' into text label 'cluster_name' using dict mapping (0->login)
df1['cluster_name'] = df1['cluster'].map(dict2)

df1.head()

,Number,Assignment_group,Assigned_to,Priority,State,Customer,Category,Subcategory,Severity,Closure_CI,...,mon_nu,month,dom,week,week2,newtext,cluster,cluster1,cluster_name1,cluster_name
0,INC0489304,DH-Enterprise IT Service Cntr,Anil Auluri,4 - Low,Closed,Suresh Kumar Manickavel,Security,NaN,3 - Low,Network Account Issue,...,10,October,14,2,2,Network Password Reset Network ||Unable to log...,14,4,login,login
1,INC0489303,DHE-Command Center,Irving Tryon,2 - High,Closed,OMI Integration Service,Server,Windows,3 - Low,phx-vapp-835,...,10,October,14,2,2,'10.248.100.25': Ping packet good percentage i...,44,41,socket issue,cerner
2,INC0489302,DHE-Command Center,Steven Calabro,3 - Moderate,Closed,OMI Integration Service,Server,Windows,3 - Low,phx-vprt-002,...,10,October,14,2,2,'phx-vprt-002.chw.edu':CPU Utilization is '100...,7,1,utilization,utilization
3,INC0489301,DH-Enterprise IT Service Cntr,Barsha Acharya,4 - Low,Closed,Cheryl Anthony,Application,NaN,3 - Low,Cerner Millennium FirstNet 2015.10.19,...,10,October,14,2,2,cerner||Unable to access the correct view|| ce...,37,25,access,access issue
4,INC0489300,DHE-Command Center,Steven Calabro,3 - Moderate,Closed,OMI Integration Service,Server,Windows,3 - Low,bmh-vapp-002,...,10,October,14,2,2,'bmh-vapp-002.chw.edu':CPU Utilization is '95'...,7,1,utilization,utilization


##
Experiments 

Clustering by Contact Type -  Seperate event and non events and cluster them seperately
or Clustering by categories ?

#Security - Top terms

In [117]:
terms_map.head()

,terms,key
0,"login user, login user id, login, unable login...",login user
1,"cerner, issue, unable, ms, id, login, user, ac...",cerner
2,"dragon, dragon medical, dragon application iss...",dragon
3,"physician issue, cerner physician issue, cerne...",physician issue
4,"job completed, active job, completed exit, com...",job completed


In [61]:
#Classic Kmean
from sklearn.cluster import KMeans

num_clusters = 50
km2 = KMeans(n_clusters=num_clusters,init='k-means++', max_iter=100, n_init=1)
#km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,init_size=1000, batch_size=1000,
%time km2.fit(tfidf_matrix)
clusters = km2.labels_.tolist()



KeyboardInterrupt: 

AttributeError: 'KMeans' object has no attribute 'labels_'

In [118]:
#df1['cluster'].value_counts()
df3['cluster'].value_counts()

1    11338
0      857
8      183
3      172
2      162
5      144
6      143
7       98
4       91
9       15
Name: cluster, dtype: int64

In [ ]:
Todo :  Visual ngram feature embedding

Issues
Manual checking the clustered data, since we have event data coming into login pwd cluster which should not be 


# Other methods
